dataset: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2
- click Export 
- choose 'CSV'

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.getOrCreate()
spark

# Preprocess Chicago's reported Crimes Data

In [6]:
from pyspark.sql.functions import to_timestamp, col, lit

In [ ]:
rc = spark.read.csv('.')